In [ ]:
# Shelterluv Adoption & Length of Stay Analysis
# ---------------------------------------------
# Basic setup: load libraries and configure visuals

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Make plots look nice
sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# Show all columns in pandas output
pd.set_option("display.max_columns", None)